In [ ]:
#sub49のnotebook

In [ ]:
#!pip install optuna

In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 14.6 MB/s 


In [ ]:
!pip install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=cbd79ea31c070a5ccd3967a33c8b5b76114ceb4238dc02bc2c61ff50fd774cfa
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [ ]:
!pip install crs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
 

In [ ]:
class config:
    run_type = 'colab'
    random_seed = 2022
    nfold = 5
    target = 'pm25_mid'

In [ ]:
import os
import re
from IPython.lib import kernel
from requests import get

if config.run_type == 'colab':
    from google.colab import drive, files
    
    drive.mount('/content/drive')
    #ご自身のフォルダパスを指定
    Drive = "/content/drive/My Drive/python/sony/"
    Exp = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'][:-6]
    Exp = re.sub('.ip', '', Exp)
        
Input = os.path.join(Drive, 'input')
Output = os.path.join(Drive, 'output')
Output_Exp = os.path.join(Output, Exp)
Output_Model = os.path.join(Output_Exp, "model")
Output_Preds = os.path.join(Output_Exp, "preds")
Output_Feats = os.path.join(Output_Exp, "feats")

Submission = os.path.join(Drive, "submission")

dirs = [Output_Model, Output_Preds, Output_Feats]
for d in dirs:
    if not os.path.isdir(d):
        os.makedirs(d, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import gc
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import datetime

from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import optuna.integration.lightgbm as opt_lgb

pd.set_option('max_rows', 400)
pd.set_option('max_columns', 100)

In [ ]:
train_data = pd.read_csv(os.path.join(Input, 'train.csv'))\
               .assign(timestamp=lambda d: pd.to_datetime(d['year'].astype(str) + '-' + d['month'].astype(str) + '-' + d['day'].astype(str)))
test_data = pd.read_csv(os.path.join(Input, 'test.csv'))\
               .assign(timestamp=lambda d: pd.to_datetime(d['year'].astype(str) + '-' + d['month'].astype(str) + '-' + d['day'].astype(str)))

In [ ]:
df_all = pd.concat([train_data,test_data],sort=False)

In [ ]:
import utm
from pyproj import Proj
import crs as CRS

lon = df_all['lon']
lat = df_all['lat']

df = pd.DataFrame({'lng': lon, 'lat': lat})

list_utm = []
xy_meters = [utm.from_latlon(lat,long) for (lat, long) in zip(df['lat'],df['lng'])]
for row in xy_meters:
    list_utm.append(row[2:4])

'''
list_utm = []
for lat,lon in zip(df['lat'], df['lon']):
    e2u_zone=int(divmod(lat, 6)[0])+31
    e2u_conv=Proj(proj='utm', zone=e2u_zone, ellps='WGS84')
   
    utmx, utmy=e2u_conv(lat, lon)
    list_utm.append(e2u_zone)

if lat<0:
    utmy=utmy+10000000

print(" UTM zone is ", e2u_zone, " \n", \
      "UTM Easting is", utmx, "[m]\n",\
      "UTM Northing is ", utmy, "[m]")
'''

ModuleNotFoundError: ignored

In [ ]:
print(list_utm)

In [ ]:
lst = list_utm
df_zone = pd.DataFrame(lst, columns = ['utm_zone_x','utm_zone_y'])

In [ ]:
df_zone.to_csv('drive/My Drive/python/sony/input/zone.csv', index=False)

In [ ]:
df_all = df_all.reset_index()

In [ ]:
df_all = pd.concat([df_all, df_zone], axis=1)
df_all = df_all.drop('index',axis=1)

In [ ]:
df_all['utm_zone'] = (df_all['utm_zone_x']).astype(str) + df_all['utm_zone_y']

In [ ]:
print(df_all)

In [ ]:
train_data = df_all.iloc[:len(train_data)]
test_data = df_all.iloc[len(train_data):].reset_index(drop=True)

In [ ]:
def make_corr_array(df, cols):
    output = pd.DataFrame(1 - squareform(pdist(df[cols].T, 'correlation')),
                          columns=cols, index=cols)
    return output

In [ ]:
corr_df = make_corr_array(train_data, [c for c in train_data.columns if re.search(r'mid|min|max|cnt|var|lat|lon', c)])

In [ ]:
corr_top10 = corr_df.loc[:, ['pm25_mid']].assign(abs_value=lambda d: np.abs(d['pm25_mid']))\
                    .sort_values('abs_value', ascending=False)\
                    .iloc[1:11, :].index.tolist()

In [ ]:
class AbstractBaseBlock:
    def fit(self, input_df: pd.DataFrame, y=None):
        return self.transform(input_df)
    
    def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
        raise NotImplementedError()

In [ ]:
class NumericFeatBlock(AbstractBaseBlock):
    def __init__(self, col: str):
        self.col = col
        
    def fit(self, input_df, y=None):
        pass
        
    def transform(self, input_df):
        return input_df.loc[:, self.col]

In [ ]:
class CategoricalFeatBlock(AbstractBaseBlock):
    def __init__(self, col: str, whole_df = None, threshold=0.001, is_label=True, is_dummy=False):
        self.col = col
        self.whole_df = whole_df
        self.threshold = threshold
        self.is_label = is_label
        self.is_dummy = is_dummy
    
    def fit(self, input_df, y=None):
        if self.whole_df == None:
            df = input_df.loc[:, self.col]
        else:
            df = self.whole_df.loc[:, self.col]
        vc = df.value_counts(normalize=True).reset_index()
        vc = vc.assign(thresh=lambda d: np.where(d[self.col].values >= self.threshold, 1, 0))\
               .assign(thresh=lambda d: d['thresh'].cumsum() - d['thresh'])
        self.label_dict_ = dict(vc[['index', 'thresh']].values)
        self.label_other_ = np.max(self.label_dict_.values())
        
        return self.transform(input_df)
        
    def transform(self, input_df):
        out_df = pd.DataFrame()
        label_df = pd.DataFrame()
        label_df[f'{self.col}_label_enc'] = np.vectorize(lambda x: self.label_dict_.get(x, self.label_other_))\
                                                        (input_df[self.col].values)
        if self.is_label:
            out_df = pd.concat([out_df, label_df], axis=1)
            
        if self.is_dummy:
            label_df[f'{self.col}_label_enc'] = label_df[f'{self.col}_label_enc'].astype(object)
            out_df = pd.concat([out_df, pd.get_dummies(label_df)], axis=1)
        
        return out_df

In [ ]:
class DateFeatureBlock(AbstractBaseBlock):
    def __init__(self, is_get_weekday=True):
        self.is_get_weekday = is_get_weekday
        
    def fit(self, input_df, y=None):
        pass
        
    def transform(self, input_df):
        out_df = pd.DataFrame()
        out_df = input_df[['year', 'month', 'day']]
        if self.is_get_weekday:
            out_df = out_df.assign(weekday=[x.dayofweek for x in input_df['timestamp'].tolist()])
        
        return out_df

In [ ]:
class AggregateValueBlock(AbstractBaseBlock):
    def __init__(self, key_col, agg_dict, whole_df=None):
        self.key_col = key_col
        self.agg_dict = agg_dict
        self.whole_df = whole_df
        
    def fit(self, input_df, y=None):
        if self.whole_df == None:
            df = input_df
        else:
            df = self.whole_df
        agg_df = df.groupby(self.key_col).agg(self.agg_dict)
        agg_df.columns = ['_'.join(c) for c in agg_df.columns]
        self.agg_df_ = agg_df.add_prefix('_')\
                             .add_prefix('_'.join(self.key_col))\
                             .add_prefix('agg_').reset_index()
        
        return self.transform(input_df)
    
    def transform(self, input_df):
        out_df = pd.DataFrame()
        out_df = pd.merge(input_df[self.key_col], self.agg_df_, how='left', on=self.key_col)\
                   .drop(self.key_col, axis=1)\
                   .fillna(0)
        
        return out_df

In [ ]:
numeric_columns = [c for c in train_data.columns if re.search(r'max|mid|min|cnt|var|lat|lon', c) and not re.search(r'pm25', c)]
categorical_columns = ['Country','utm_zone']
agg_country_value = dict([(c, [np.mean, np.max, np.min, np.std, np.var]) for c in corr_top10])

In [ ]:
run_blocks = [
    *[NumericFeatBlock(c) for c in [numeric_columns]],
    *[CategoricalFeatBlock(c) for c in categorical_columns],
    *[DateFeatureBlock()],
    *[AggregateValueBlock(['utm_zone', 'month'], agg_country_value)],    
]

In [ ]:
def get_train_data(input_df, feat_blocks, y=None, fit_df=None):
    if fit_df is None:
        fit_df = input_df.copy()
        
    for block in feat_blocks:
        block.fit(fit_df, y)
        
    out = [block.transform(input_df) for block in feat_blocks]
    out = pd.concat(out, axis=1)
    
    return out

def get_test_data(input_df, feat_blocks):
    
    out = [block.transform(input_df) for block in feat_blocks]
    out = pd.concat(out, axis=1)
    
    return out

In [ ]:
df_train = get_train_data(train_data, run_blocks, fit_df=pd.concat([train_data, test_data], ignore_index=True))
df_train

In [ ]:
df_test = get_test_data(test_data, run_blocks)
df_test

In [ ]:
df_all = pd.concat([df_train,df_test],sort=False)

In [ ]:

#df_all['diff_no2_mid_aggmean'] = df_all['agg_Country_month_no2_mid_mean'] - df_all['no2_mid']
#df_all['diff_no2_mid_aggamax'] = df_all['agg_Country_month_no2_mid_amax'] - df_all['no2_mid']
df_all['diff_no2_mid_aggamin'] = df_all['agg_utm_zone_month_no2_mid_amin'] - df_all['no2_mid']

#df_all['diff_no2_mid_aggminmax'] = df_all['diff_no2_mid_aggamax'] - df_all['diff_no2_mid_aggamin']
#df_all['ratio_no2_mid_aggminmax_mean'] = df_all['diff_no2_mid_aggminmax'] / df_all['diff_no2_mid_aggmean']

#df_all['diff_no2_max_aggmean'] = df_all['agg_Country_month_no2_max_mean'] - df_all['no2_max']
#df_all['diff_no2_max_aggamax'] = df_all['agg_Country_month_no2_max_amax'] - df_all['no2_max']
#df_all['diff_no2_max_aggamin'] = df_all['agg_Country_month_no2_max_amin'] - df_all['no2_max']

#df_all['diff_no2_max_aggminmax'] = df_all['diff_no2_max_aggamax'] - df_all['diff_no2_max_aggamin']
#df_all['ratio_no2_max_aggminmax_mean'] = df_all['diff_no2_max_aggminmax'] / df_all['diff_no2_max_aggmean']

#df_all['diff_no2_min_aggmean'] = df_all['agg_Country_month_no2_min_mean'] - df_all['no2_min']
#df_all['diff_no2_min_aggamax'] = df_all['agg_Country_month_no2_min_amax'] - df_all['no2_min']
#df_all['diff_no2_min_aggamin'] = df_all['agg_Country_month_no2_min_amin'] - df_all['no2_min']

#df_all['diff_no2_min_aggminmax'] = df_all['diff_no2_min_aggamax'] - df_all['diff_no2_min_aggamin']
#df_all['ratio_no2_min_aggminmax_mean'] = df_all['diff_no2_min_aggminmax'] / df_all['diff_no2_min_aggmean']

#df_all['diff_no2_minmax_aggmean'] = df_all['diff_no2_max_aggmean'] -df_all['diff_no2_min_aggmean']
#df_all['diff_no2_minmax_aggamax'] = df_all['diff_no2_max_aggamax'] -df_all['diff_no2_min_aggamax']
#df_all['diff_no2_minmax_aggamin'] = df_all['diff_no2_max_aggamin'] -df_all['diff_no2_min_aggamin']

#df_all['diff_co_mid_aggmean'] = df_all['agg_Country_month_co_mid_mean'] - df_all['co_mid']
#df_all['diff_co_mid_aggamax'] = df_all['agg_Country_month_co_mid_amax'] - df_all['co_mid']
df_all['diff_co_mid_aggamin'] = df_all['agg_utm_zone_month_co_mid_amin'] - df_all['co_mid']

#df_all['diff_co_mid_aggminmax'] = df_all['diff_co_mid_aggamax'] - df_all['diff_co_mid_aggamin']
#df_all['ratio_co_mid_aggminmax_mean'] = df_all['diff_co_mid_aggminmax'] / df_all['diff_co_mid_aggmean']

#df_all['diff_co_max_aggmean'] = df_all['agg_Country_month_co_max_mean'] - df_all['co_max']
#df_all['diff_co_max_aggamax'] = df_all['agg_Country_month_co_max_amax'] - df_all['co_max']
df_all['diff_co_max_aggamin'] = df_all['agg_utm_zone_month_co_max_amin'] - df_all['co_max']

#df_all['diff_co_max_aggminmax'] = df_all['diff_co_max_aggamax'] - df_all['diff_co_max_aggamin']
#df_all['ratio_co_max_aggminmax_mean'] = df_all['diff_co_max_aggminmax'] / df_all['diff_co_max_aggmean']

#df_all['diff_co_min_aggmean'] = df_all['agg_Country_month_co_min_mean'] - df_all['co_min']
#df_all['diff_co_min_aggamax'] = df_all['agg_Country_month_co_min_amax'] - df_all['co_min']
#df_all['diff_co_min_aggamin'] = df_all['agg_Country_month_co_min_amin'] - df_all['co_min']
'''

#df_all['diff_co_min_aggminmax'] = df_all['diff_co_min_aggamax'] - df_all['diff_co_min_aggamin']
#df_all['ratio_co_min_aggminmax_mean'] = df_all['diff_co_min_aggminmax'] / df_all['diff_co_min_aggmean']

#df_all['diff_co_minmax_aggmean'] = df_all['diff_co_max_aggmean'] -df_all['diff_co_min_aggmean']
#df_all['diff_co_minmax_aggamax'] = df_all['diff_co_max_aggamax'] -df_all['diff_co_min_aggamax']
#df_all['diff_co_minmax_aggamin'] = df_all['diff_co_max_aggamin'] -df_all['diff_co_min_aggamin']

#df_all['diff_so2_max_aggmean'] = df_all['agg_Country_month_so2_max_mean'] - df_all['so2_max']
#df_all['diff_so2_max_aggamax'] = df_all['agg_Country_month_so2_max_amax'] - df_all['so2_max']
df_all['diff_so2_max_aggamin'] = df_all['agg_Country_month_so2_max_amin'] - df_all['so2_max']

#df_all['diff_so2_max_aggminmax'] = df_all['diff_so2_max_aggamax'] - df_all['diff_so2_max_aggamin']
#df_all['ratio_so2_max_aggminmax_mean'] = df_all['diff_so2_max_aggminmax'] / df_all['diff_so2_max_aggmean']

#df_all['diff_so2_mid_aggmean'] = df_all['agg_Country_month_so2_mid_mean'] - df_all['so2_mid']
#df_all['diff_so2_mid_aggamax'] = df_all['agg_Country_month_so2_mid_amax'] - df_all['so2_mid']
df_all['diff_so2_mid_aggamin'] = df_all['agg_Country_month_so2_mid_amin'] - df_all['so2_mid']

#df_all['diff_so2_mid_aggminmax'] = df_all['diff_so2_mid_aggamax'] - df_all['diff_so2_mid_aggamin']
#df_all['ratio_so2_mid_aggminmax_mean'] = df_all['diff_so2_mid_aggminmax'] / df_all['diff_so2_mid_aggmean']

df_all['diff_o3_max_aggmean'] = df_all['agg_Country_month_o3_max_mean'] - df_all['o3_max']
#df_all['diff_o3_max_aggamax'] = df_all['agg_Country_month_o3_max_amax'] - df_all['o3_max']
df_all['diff_o3_max_aggamin'] = df_all['agg_Country_month_o3_max_amin'] - df_all['o3_max']

df_all['diff_o3_max_aggminmax'] = df_all['diff_o3_max_aggamax'] - df_all['diff_o3_max_aggamin']
df_all['ratio_o3_max_aggminmax_mean'] = df_all['diff_o3_max_aggminmax'] / df_all['diff_o3_max_aggmean']

df_all['diff_ws_mid_aggmean'] = df_all['agg_Country_month_ws_mid_mean'] - df_all['ws_mid']
df_all['diff_ws_mid_aggamax'] = df_all['agg_Country_month_ws_mid_amax'] - df_all['ws_mid']
df_all['diff_ws_mid_aggamin'] = df_all['agg_Country_month_ws_mid_amin'] - df_all['ws_mid']

df_all['diff_ws_mid_aggminmax'] = df_all['diff_ws_mid_aggamax'] - df_all['diff_ws_mid_aggamin']
df_all['ratio_ws_mid_aggminmax_mean'] = df_all['diff_ws_mid_aggminmax'] / df_all['diff_ws_mid_aggmean']
'''

In [ ]:
import wxparams as wx
'''
df_all['mix_ratio'] = wx.Mixing_Ratio(df_all['dew_mid'], df_all['pressure_mid'], formula="Bolton")
df_all['spec_humidity'] = wx.Specific_Humidity(df_all['dew_mid'], df_all['pressure_mid'], formula="Bolton")
df_all['abs_humidity'] = wx.Absolute_Humidity(df_all['temperature_mid'], df_all['dew_mid'], formula="Bolton")
df_all['vir_temperature'] = wx.Virtual_Temperature(df_all['temperature_mid'], df_all['dew_mid'], df_all['pressure_mid'], formula="Bolton")
'''
df_all['theta'] = wx.Theta(df_all['temperature_mid'], df_all['pressure_mid'])
df_all['tlcl'] = wx.Tlcl(df_all['temperature_mid'], df_all['dew_mid'])
'''
df_all['theta_e'] = wx.Theta_e(df_all['temperature_mid'], df_all['dew_mid'], df_all['pressure_mid'], formula="Bolton")
df_all['di'] = 0.81*df_all['temperature_mid']+0.01*df_all['humidity_mid']*(0.99*df_all['temperature_mid']-14.3)+4
'''

df_all['co_minmax'] = df_all['co_max'] - df_all['co_min']
df_all['o3_minmax'] = df_all['o3_max'] - df_all['o3_min']
df_all['so2_minmax'] = df_all['so2_max'] - df_all['so2_min']
df_all['no2_minmax'] = df_all['no2_max'] - df_all['no2_min']

df_all['temperature_minmax'] = df_all['temperature_max'] - df_all['temperature_min']
df_all['humidity_minmax'] = df_all['humidity_max'] - df_all['humidity_min']
df_all['pressure_minmax'] = df_all['pressure_max'] - df_all['pressure_min']
df_all['ws_minmax'] = df_all['ws_max'] - df_all['ws_min']
df_all['dew_minmax'] = df_all['dew_max'] - df_all['dew_min']


df_all['co_mean_mid'] = df_all['co_min'] + df_all['co_minmax']/2 - df_all['co_mid']
df_all['o3_mean_mid'] = df_all['o3_min'] + df_all['o3_minmax']/2 - df_all['o3_mid']
df_all['so2_mean_mid'] = df_all['so2_min'] + df_all['so2_minmax']/2 - df_all['so2_mid']
df_all['no2_mean_mid'] = df_all['no2_min'] + df_all['no2_minmax']/2 - df_all['no2_mid']

df_all['temperature_mean_mid'] = df_all['temperature_min'] + df_all['temperature_minmax']/2 - df_all['temperature_mid']
df_all['humidity_mean_mid'] = df_all['humidity_min'] + df_all['humidity_minmax']/2 - df_all['humidity_mid']
df_all['pressure_mean_mid'] = df_all['pressure_min'] + df_all['pressure_minmax']/2 - df_all['pressure_mid']
df_all['ws_mean_mid'] = df_all['ws_min'] + df_all['ws_minmax']/2 - df_all['ws_mid']

df_all['co_humidity'] = df_all['co_mid']*df_all['humidity_mid']/100
df_all['o3_humidity'] = df_all['o3_mid']*df_all['humidity_mid']/100
df_all['so2_humidity'] = df_all['so2_mid']*df_all['humidity_mid']/100
df_all['no2_humidity'] = df_all['no2_mid']*df_all['humidity_mid']/100
'''
df_all['co_ws'] = df_all['co_mid']*df_all['ws_mid']
df_all['o3_ws'] = df_all['o3_mid']*df_all['ws_mid']
df_all['so2_ws'] = df_all['so2_mid']*df_all['ws_mid']
df_all['no2_ws'] = df_all['no2_mid']*df_all['ws_mid']

df_all['temperature_humidity'] = df_all['temperature_mid']*df_all['humidity_mid']

df_all['cnt'] = df_all['co_cnt']*df_all['o3_cnt']*df_all['so2_cnt']*df_all['no2_cnt']*df_all['temperature_cnt']*df_all['humidity_cnt']*df_all['pressure_cnt']*df_all['ws_cnt']
'''

In [ ]:

def encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df

df_all = encode(df_all, 'weekday')
df_all = encode(df_all, 'day')
df_all = encode(df_all, 'month')
df_all = encode(df_all, 'year')


In [ ]:
'''
group =['Country_label_enc']
feature_cols = ['co_mid', 'no2_mid']

outputs = [df_all]
grp_df = df_all.groupby(group)[feature_cols]

for lag in [-3,-2, -1, 1, 2,3]:
   # shift
   outputs.append(grp_df.shift(lag).add_prefix(f'shift{lag}_'))
   # diff
   outputs.append(grp_df.diff(lag).add_prefix(f'diff{lag}_'))


# rolling
for window in [3,30]:
    tmp_df = grp_df.rolling(window, min_periods=1)
    # 移動平均を取る
    tmp_df = tmp_df.mean().add_prefix(f'rolling{window}_mean_')
    outputs.append(tmp_df.reset_index(drop=True))


df_all = df_all.loc[~df_all.index.duplicated(keep='first')]
df_all = pd.concat(outputs,axis=1)

#https://stackoverflow.com/questions/71240439/reindexing-only-valid-with-uniquely-valued-index-objects
'''

In [ ]:
print(df_all)

In [ ]:
#df_all.plot.scatter('month_sin', 'month_cos').set_aspect('equal')

In [ ]:
df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):].reset_index(drop=True)

In [ ]:
target = train_data[config.target]

In [ ]:
'''
# LightGBM用のデータセットに変換
trainval = opt_lgb.Dataset(df_train, target)

# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'regression',
          'metric': 'rmse',
          'random_seed':42} 

# クロスバリデーションによるハイパーパラメータの探索 3fold
tuner = opt_lgb.LightGBMTunerCV(params, trainval, verbose_eval=100, early_stopping_rounds=100, folds=KFold(n_splits=5))

# ハイパーパラメータ探索の実行
tuner.run()

# サーチしたパラメータの表示
best_params = tuner.best_params
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))
'''

In [ ]:
def group_kfold_split(df, col_group, n_splits, random_state):

    group_id = df[col_group].value_counts().reset_index()
    group_id.columns = [col_group, 'count']
    group_id_list = group_id[col_group].unique()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kf_list = list(kf.split(group_id_list))
    cv_idx = [(df[col_group].isin(group_id_list[idx_trn]), df[col_group].isin(group_id_list[idx_val])) for idx_trn, idx_val in kf_list]

    return [(df[idx_trn].index.tolist(), df[idx_val].index.tolist()) for idx_trn, idx_val in cv_idx]

In [ ]:
#cross validation strategy
kf = KFold(n_splits=config.nfold, shuffle=True, random_state=config.random_seed)
kf_cv = list(kf.split(train_data))
gkf_cv = group_kfold_split(train_data, 'City', n_splits=config.nfold, random_state=config.random_seed)

In [ ]:
def cal_rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
def fit_lgb(x, y, cv, model_params, fit_params, fobj=None, feval=None):

    models = []
    n_records = y.shape[0]
    oof_pred = np.zeros(n_records, dtype=np.float32)
    fold = 0
    data_labels = [re.sub(r'[",\[\]{}:()]', '_', c) for c in x.columns.tolist()]

    model_params.update(deterministic = True)

    for trn_idx, val_idx in cv:

        fold += 1
        x_train, x_valid = x.iloc[trn_idx].values, x.iloc[val_idx].values
        y_train, y_valid = np.array(y.iloc[trn_idx]), np.array(y.iloc[val_idx])

        lgb_train = lgb.Dataset(x_train, y_train, feature_name=data_labels)
        lgb_valid = lgb.Dataset(x_valid, y_valid, feature_name=data_labels, reference=lgb_train)

        lgb_model = lgb.train(model_params,
                              train_set=lgb_train,
                              valid_sets=[lgb_train, lgb_valid],
                              fobj=fobj,
                              feval=feval,
                              verbose_eval=fit_params['verbose_eval'],
                              num_boost_round=fit_params['num_boost_rounds'],
                              callbacks=[lgb.early_stopping(fit_params['early_stopping_rounds'])],
                              )

        pred_valid = lgb_model.predict(x_valid, num_iteration=lgb_model.best_iteration)
        oof_pred[val_idx] = pred_valid
        models.append(lgb_model)

        print(f' - fold{fold}_RMSE - {cal_rmse(y_valid, pred_valid):4f}')

    print(f' - CV_RMSE - {cal_rmse(oof_pred, np.array(y)):4f}') 

    return oof_pred, models

In [ ]:
'''
#Params: 
    objective: regression
    metric: rmse
    random_seed: 42
    feature_pre_filter: False
    lambda_l1: 0.000338595481765331
    lambda_l2: 9.60230430368646
    num_leaves: 32
    feature_fraction: 0.6799999999999999
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 20
'''

In [ ]:
lgb_model_params = {'boosting_type': 'gbdt',
                
                    'objective': 'rmse',

                    'lambda_l1': 0.000338595481765331,
                    'lambda_l2': 9.60230430368646,
                    'feature_fraction': 0.6799999999999999,

                    'learning_rate': 0.05,
                    'max_depth': -1,
                    'num_leaves': 32,
                    'min_data_in_leaf': 20,
                    'max_bin': 255,
                    
                    'reg_lambda': 1.0,
                    'reg_alpha': 1.,
                   
                    'colsample_bytree': 0.8,
                    'subsample': 0.8,
                    'subsample_freq': 1,
                    
                    'random_state': config.random_seed,
                    'verbose': -1,
                    'n_jobs': -1,
                    }

lgb_fit_params ={'num_boost_rounds': 10000,
                 'early_stopping_rounds': 100,
                 'verbose_eval': 100,
                }

In [ ]:
oof_valid_lgb1, lgb_models1 = fit_lgb(x=df_train, y=target, cv=kf_cv,
                                      model_params=lgb_model_params, fit_params=lgb_fit_params
                                     )

In [ ]:
def visualize_importance(models, feat_train_df, file_name):
    
    feature_importance_df = pd.DataFrame()

    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importance(importance_type='gain')
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df],
                                          axis=0,
                                          ignore_index=True
                                         )
    
    mean_feature_importance = feature_importance_df.groupby('column').mean().sort_values('feature_importance', ascending=False)
    mean_feature_importance.to_csv(os.path.join(Output_Model, f'feature_importance_{file_name}.csv'), index=True)
    
    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:100]
    
    fig, ax = plt.subplots(figsize=(12, max(8, len(order) * 0.25)))
    sns.boxenplot(data=feature_importance_df,
                  x='feature_importance',
                  y='column',
                  order=order,
                  ax=ax,
                  palette='viridis',
                  orient='h'
                 )
    ax.tick_params(axis='x', rotation=90)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig, ax

In [ ]:
_ = visualize_importance(lgb_models1, df_train, 'lgb_model')

In [ ]:
def predict_test(models, df):
    out = np.array([model.predict(df) for model in models])
    out = np.mean(out, axis=0)
    
    return out

In [ ]:
pred_test = predict_test(lgb_models1, df_test)

In [ ]:
y_true = target
min_values = y_true.min()
pred_test[pred_test<min_values] = min_values

In [ ]:
#check distribution of train and test_pred data
def plot_prediction_distribution(y_true, y_pred, y_test):
    fig, ax = plt.subplots(figsize=(18, 6))
    sns.histplot(y_test, label='Test Predict', ax=ax, color='black', stat='density')
    sns.histplot(y_pred, label='Out Of Fold', ax=ax, color='C1', stat='density', alpha=0.5)
    sns.histplot(y_true, label='True Value', ax=ax, color='blue', stat='density', alpha=0.5)
    ax.legend()
    ax.grid()

In [ ]:
plot_prediction_distribution(target, oof_valid_lgb1, pred_test)

In [ ]:
submission = pd.read_csv(os.path.join(Input, 'submit_sample.csv'), header=None)
submission.iloc[:, 1] = pred_test
submission

In [ ]:
submission.to_csv(os.path.join(Submission, 'sub.csv'), index=False)